## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Add our `utils` directory to the system's `$PATH` so we can import Python files from sibling directories.

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import regex as re
from datetime import datetime
from dateutil.parser import parse

## Download

Retrieve the page

In [6]:
url = "https://www.amadorgov.org/services/covid-19/-fsiteid-1"

In [7]:
page = requests.get(url)

## Parse

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Get content well

In [9]:
content = soup.find("div", id="widget_4_3778_3488")

Get timestamp

In [10]:
date_text = [x.parent for x in soup(text=re.compile(r"Current Status Updated "))][
    0
].text

IndexError: list index out of range

In [72]:
date_text

'Current Status Updated 0ctober 1, 2021:'

In [73]:
date = date_text.replace("Current Status Updated ", "").replace(":", "")

In [74]:
date

'0ctober 1, 2021'

In [75]:
month, day, year = date.split()

In [76]:
month = month.replace("0", "O")

In [77]:
day = day.replace(",", "").zfill(2)

In [78]:
date_str = f"{month} {day}, {year}"

In [79]:
latest_date = datetime.now().strftime("%B %d, %Y")

Get table

In [80]:
table = content.find("table")

In [81]:
tbody = soup.tbody

In [82]:
city_cell = tbody.select("tr")[1].select("td")[0]

In [83]:
cities = city_cell.find_all("p")

Parse the result

In [84]:
dict_list = []

In [85]:
for row in cities:
    city_text = unicodedata.normalize("NFKD", row.get_text())
    city_list = city_text.split(":")
    clean_list = [x.strip(" ") for x in city_list]
    d = dict(area=clean_list[0], confirmed_cases=clean_list[1])
    dict_list.append(d)

Convert to dataframe

In [86]:
df = pd.DataFrame(dict_list)
df.insert(0, "county", "Amador")
df["county_date"] = latest_date

In [87]:
df["area"] = df["area"].astype(str)

Mark the current date

In [88]:
tz = pytz.timezone("America/Los_Angeles")

In [89]:
today = datetime.now(tz).date()

In [90]:
slug = "amador"

## Vet

In [91]:
try:
    assert not len(df) > 7
except AssertionError:
    raise AssertionError("Amador's area scraper has extra rows")

In [92]:
try:
    assert not len(df) < 7
except AssertionError:
    raise AssertionError("Amador's area scraper is missing rows")

## Export

In [93]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [94]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [95]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [96]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [97]:
df.to_csv(os.path.join(data_dir / slug / "timeseries.csv"), index=False)